In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Masking
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [ ]:
# Define paths
PROCESSED_DATA_PATH = os.path.abspath("../data/processed")
STATIC_FEATURE_NAMES_FILE = os.path.join(PROCESSED_DATA_PATH, "static_feature_names.json")

# Load the actual static feature names from preprocessing
with open(STATIC_FEATURE_NAMES_FILE, "r") as f:
    static_feature_names = json.load(f)

# List all .npz files in the processed folder
all_npz_files = [os.path.join(PROCESSED_DATA_PATH, f) for f in os.listdir(PROCESSED_DATA_PATH) if f.endswith('.npz')]
if not all_npz_files:
    raise FileNotFoundError(f"No .npz files found in {PROCESSED_DATA_PATH}")

# Step 1: Determine the majority shape among the datasets
shape_to_files = {}
for file in all_npz_files:
    data = np.load(file)
    X = data["X"]
    shape = X.shape  # Shape: (num_sequences, window_size, num_features)
    shape_to_files.setdefault(shape, []).append(file)

# Determine the majority shape by counting occurrences
all_shapes = [shape for shape, files in shape_to_files.items() for _ in range(len(files))]
majority_shape = Counter(all_shapes).most_common(1)[0][0]
print("Majority shape (num_sequences, window_size, num_features):", majority_shape)

# Step 2: Filter the npz files to include only those matching the majority shape
valid_npz_files = shape_to_files[majority_shape]
print("\nFiles with the majority shape:")
for file in valid_npz_files:
    print(file)


In [ ]:
# Step 3: Define a generator that yields individual sequences and targets only from the valid files
def data_generator():
    for file in valid_npz_files:
        data = np.load(file)
        X = data["X"]  # Shape: (num_sequences, window_size, num_features)
        y = data["y"]  # Shape: (num_sequences,)
        for i in range(X.shape[0]):
            yield X[i], y[i]

# Determine input shape from a sample file (use valid_npz_files)
sample_data = np.load(valid_npz_files[0])
input_shape = sample_data["X"].shape[1:]  # (window_size, num_features)
print("\nInput shape for the model:", input_shape)

In [ ]:
# Step 4: Create a tf.data.Dataset from the generator
dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_signature=(
        tf.TensorSpec(shape=input_shape, dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.float32)
    )
)

# Shuffle, batch, and prefetch the dataset for efficiency
batch_size = 32
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)



Majority shape (num_sequences, window_size, num_features): (492, 10, 144)

Files with the majority shape:
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\AAPL_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ABBV_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ABNB_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ABT_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ACGL_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ACN_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ADBE_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ADI_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ADM_data.npz
c:\Users\Connor\Documents\GitHub\stock-market-forecasting\data\processed\ADP_data.npz
c:\Users\Connor\Documents\Git

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 10, 144)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        53,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,569 (209.25 KB)

 Trainable params: 53,569 (209.25 KB)

 Non-trainable params: 0 (0.00 B)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
7657/7657 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 0.0392 - mae: 0.1308 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/10


C:\Users\Connor\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


7657/7657 ━━━━━━━━━━━━━━━━━━━━ 60s 8ms/step - loss: 0.0266 - mae: 0.1096 - learning_rate: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/10
7657/7657 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 0.0263 - mae: 0.1089 - learning_rate: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/10
7657/7657 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 0.0261 - mae: 0.1081 - learning_rate: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 5/10
7657/7657 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 0.0258 - mae: 0.1076 - learning_rate: 0.0010

Epoch 6: LearningRateScheduler setting learning rate to 0.0005000000237487257.
Epoch 6/10
7657/7657 ━━━━━━━━━━━━━━━━━━━━ 60s 8ms/step - loss: 0.0253 - mae: 0.1062 - learning_rate: 5.0000e-04

Epoch 7: LearningRateScheduler setting learning rate to 0.0005000000237487257.
Epoch 7/10
7657/7657 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 0.02

In [ ]:
# Step 5: Build the LSTM model
model = Sequential([
    Input(shape=input_shape),
    # Optional: Masking layer in case you pad sequences with zeros
    Masking(mask_value=0.0),
    # LSTM layer to process the time-series data
    LSTM(64, return_sequences=False),
    # Final dense layer for regression (predicting a single value)
    Dense(1)
])
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()

In [ ]:
# Step 6: Define callbacks for training
def lr_schedule(epoch, lr):
    # Example: decay learning rate every 5 epochs
    if epoch % 5 == 0 and epoch:
        return lr * 0.5
    return lr

callbacks = [
    ModelCheckpoint("../model/best_model.keras", save_best_only=True, monitor="loss", mode="min"),
    LearningRateScheduler(lr_schedule, verbose=1)
]

# Train the model
num_epochs = 10
history = model.fit(dataset, epochs=num_epochs, callbacks=callbacks)